## Section B - Working with DataFrames and SQL

In [4]:
import numpy as np
import pandas as pd
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark_session = SparkSession.builder\
        .master("spark://192.168.2.250:7077") \
        .appName("Nikitha_Krishnamurthy_A3_SQL")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 6)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

sc = spark_session.sparkContext

sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/20 19:31:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
sqlContext = SQLContext(spark_session.sparkContext)
sqlContext

/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


#### B.1 Load the CSV file from HDFS, and call show() to verify the data is loaded correctly.

In [7]:
data_1= sqlContext.read.csv('hdfs://192.168.2.250:9000/parking-citations.csv',
                        header='true', inferSchema='true').cache() #load the cvs file from HDFS using spark API 

data_1.show(2)#verifying if the file loaded and showing up correctly


+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+---------------+-----+------+--------------+---------------------+-----------+--------+---------+------------------+-----------------+----------------------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|       Location|Route|Agency|Violation code|Violation Description|Fine amount|Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+---------------+-----+------+--------------+---------------------+-----------+--------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    NULL|       NULL|            CA|         200304.0|NULL|HOND|        PA|   GY|13147 WELBY WAY|01

### B.2 Print the schema for the DataFrame.

In [8]:
data_1.printSchema() #print the schema

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



### B.3 Count the number of rows in the CSV file.

In [9]:
data_1.count() #couting the number of rows

13077724

### B.4 Count the number of partitions in the underlying RDD

In [11]:
data_1.rdd.getNumPartitions() #counting number of partitions

16

### B.5 Drop the columns VIN, Latitude and Longitude.

In [12]:
drop_columns = ['VIN','Latitude','Longitude']
data_1=data_1.drop(*drop_columns) #dropping the columns mentioned in the list
data_1.printSchema()#printing to verify if the columns are dropped

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: double (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



### B.6 Find the maximum fine amount. How many fines have this amount? You need to convert the ‘fine amount’ column to a float to do this correctly.

In [14]:
#converting datatype of "Fine amount" to float
data_new=data_1.withColumn("Fine amount",col("Fine amount").cast("float"))
data_new.printSchema() #printing to verify the datatype

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: float (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [15]:

#to get maximum amount fine charged
data_new.agg(max("Fine amount")).show()

#showing the number of times the maximum fine is charged
data_new.groupBy('Fine amount').count().sort("Fine amount",ascending=False).show(1)



+----------------+
|max(Fine amount)|
+----------------+
|          1100.0|
+----------------+



+-----------+-----+
|Fine amount|count|
+-----------+-----+
|     1100.0|  626|
+-----------+-----+
only showing top 1 row



### B.7 Show the top 20 most frequent vehicle makes, and their frequencies.

In [17]:
#counting the number of most frequent Make
data_new.groupBy('Make').count().orderBy(col("count").desc()).show()

+----+-------+
|Make|  count|
+----+-------+
|TOYT|2150768|
|HOND|1479996|
|FORD|1116235|
|NISS| 945133|
|CHEV| 892676|
| BMW| 603092|
|MERZ| 543298|
|VOLK| 432030|
|HYUN| 404917|
|DODG| 391686|
|LEXS| 368420|
| KIA| 328155|
|JEEP| 316300|
|AUDI| 255395|
|MAZD| 242344|
|OTHR| 205546|
| GMC| 184889|
|INFI| 174315|
|CHRY| 159948|
|SUBA| 154640|
+----+-------+
only showing top 20 rows



### B.8 Let’s expand some abbreviations in the color column. Create a User Defined Function to create a new column, ‘colorlong’, mapping the original colors to their corresponding values in the dictionary below. If there is no key matching the original color, use the original color.

In [21]:
#function to map the old color value with new abbreviated values given
def convert(dict):
    return udf(lambda x : dict.get(x), StringType())

COLORS = {
'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White', 'WH':'White',
'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
}

data_new=data_new.withColumn("color long", convert(COLORS)("Color"))#adding new column named as "color long"
data_new.show()#printing the table to check if column is added

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+----------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|Agency Description|Color Description|Body Style Description|color long|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+----------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    NULL|       NULL|            CA|         200304.0|HOND|        PA|   GY|     13147 WELBY WAY|01521|   1.0|        4000A

In [20]:
data_new.printSchema()#verifying to check if column is added

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: float (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)
 |-- color long: string (nullable = true)



### B.9 Using this new column, what’s the most frequent colour value for Toyotas (TOYT)?

In [23]:
#printing the most frequent color value of Make given
data_new.groupBy('Make').agg(mode("color long")).where(data_new["Make"]=="TOYT").show()

+----+----------------+
|Make|mode(color long)|
+----+----------------+
|TOYT|            Gray|
+----+----------------+



In [17]:
spark_session.stop()